In [70]:
from datetime import datetime

In [2]:

from pyspark.sql.types import StructType, StructField, FloatType, LongType, StringType

feats = []
f = open('features.txt')
for line_num, line in enumerate(f):
    if line_num == 0:
        # Timestamp
        feats.append(StructField(line.strip(), LongType(), True))
    elif line_num == 1:
        # Geohash
        feats.append(StructField(line.strip(), StringType(), True))
    else:
        # Other features
        feats.append(StructField(line.strip(), FloatType(), True))
    
schema = StructType(feats)

print(schema)

StructType(List(StructField(Timestamp,LongType,true),StructField(Geohash,StringType,true),StructField(geopotential_height_lltw,FloatType,true),StructField(water_equiv_of_accum_snow_depth_surface,FloatType,true),StructField(drag_coefficient_surface,FloatType,true),StructField(sensible_heat_net_flux_surface,FloatType,true),StructField(categorical_ice_pellets_yes1_no0_surface,FloatType,true),StructField(visibility_surface,FloatType,true),StructField(number_of_soil_layers_in_root_zone_surface,FloatType,true),StructField(categorical_freezing_rain_yes1_no0_surface,FloatType,true),StructField(pressure_reduced_to_msl_msl,FloatType,true),StructField(upward_short_wave_rad_flux_surface,FloatType,true),StructField(relative_humidity_zerodegc_isotherm,FloatType,true),StructField(categorical_snow_yes1_no0_surface,FloatType,true),StructField(u-component_of_wind_tropopause,FloatType,true),StructField(surface_wind_gust_surface,FloatType,true),StructField(total_cloud_cover_entire_atmosphere,FloatType,tru

In [3]:

#df = spark.read.format('csv').option('sep', '\t').schema(schema).load('/Volumes/evo/Datasets/NAM_2015_S/*')
df = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:40910/nam_tiny.tdv')
# df = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:40910/datasets/nam_201501.tdv.gz')


In [4]:
df_large = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:40910/datasets/nam_201509.tdv.gz')


In [32]:
df_entire = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:40910/datasets/*')


In [5]:
really_hot = df.filter(df.snow_depth_surface != 0).count()
print(really_hot)

hot_and_humid = df.filter(df.temperature_surface > 313).filter(df.relative_humidity_zerodegc_isotherm > .8).count()
print(hot_and_humid)

25
0


In [8]:
# df.filter(df.snow_depth_surface != 0).take(5)

[Row(Timestamp=1426377600000, Geohash='cf7ecr4h2ps0', geopotential_height_lltw=136.53125, water_equiv_of_accum_snow_depth_surface=77.0, drag_coefficient_surface=0.0, sensible_heat_net_flux_surface=-39.57763671875, categorical_ice_pellets_yes1_no0_surface=0.0, visibility_surface=24221.587890625, number_of_soil_layers_in_root_zone_surface=3.0, categorical_freezing_rain_yes1_no0_surface=0.0, pressure_reduced_to_msl_msl=99602.0, upward_short_wave_rad_flux_surface=6.625, relative_humidity_zerodegc_isotherm=34.0, categorical_snow_yes1_no0_surface=0.0, u-component_of_wind_tropopause=27.527877807617188, surface_wind_gust_surface=16.158788681030273, total_cloud_cover_entire_atmosphere=100.0, upward_long_wave_rad_flux_surface=314.0560302734375, land_cover_land1_sea0_surface=1.0, vegitation_type_as_in_sib_surface=18.0, v-component_of_wind_pblri=12.31915283203125, albedo_surface=38.75, lightning_surface=0.0, ice_cover_ice1_no_ice0_surface=0.0, convective_inhibition_surface=-0.65234375, pressure_su

# Warn_up

## Q1. Unknown Feature: Choose a feature from the data dictionary above that you have never heard of before. Inspect some of the values for the feature (such as its average, min, max, etc.) and try to guess what it measures. Was your hypothesis correct?

In [9]:
from pyspark.sql.functions import avg

df.select(avg(df.geopotential_height_pblri)).show()

+------------------------------+
|avg(geopotential_height_pblri)|
+------------------------------+
|              406.451460521314|
+------------------------------+



This attribute may records geopotential height for each loaction.

## Q2. Hot hot hot: When and where was the hottest temperature observed in the dataset? Is it an anomaly?

In [64]:
# Creating an SQL 'table'
df_entire.createOrReplaceTempView("TEMP_DF")

# Let's get all the snow cover values:
hot = spark.sql("SELECT Geohash,temperature_surface FROM TEMP_DF WHERE temperature_surface IN (SELECT MAX(temperature_surface) FROM TEMP_DF) ").collect()

for x in hot:
    print(x)


Row(Geohash='d5dpds10m55b', temperature_surface=331.390625)


The hottest temperature in this dataset is 331.39F (166.3C). Obviously, Human cannot live in such hot place.

## Q3. So Snowy: Find a location that is snowy all year (there are several). Locate a nearby town/city and provide a small writeup about it. Include pictures if you’d like.

In [ ]:
# Creating an SQL 'table'
df_large.createOrReplaceTempView("TEMP_DF")

# Let's get all the snow cover values:
snow = spark.sql("SELECT Geohash FROM TEMP_DF WHERE Geohash NOT IN (SELECT DISTINCT Geohash FROM TEMP_DF WHERE categorical_snow_yes1_no0_surface = 0) ").collect()
for x in snow:
    print(x)

In [123]:
# Creating an SQL 'table'
df_entire.createOrReplaceTempView("TEMP_DF")
started_at = datetime.now()
# Let's get all the snow cover values:
# not_snow_all_year = spark.sql("SELECT DISTINCT Geohash FROM TEMP_DF WHERE categorical_snow_yes1_no0_surface = 0 ").collect()
# all_Geohash = spark.sql("SELECT DISTINCT Geohash FROM TEMP_DF").collect()
# snow = spark.sql("SELECT DISTINCT Geohash from TEMP_DF group by Geohash having avg(snow_cover_surface) >= 100").collect()
snow = spark.sql("SELECT DISTINCT Geohash, avg(categorical_snow_yes1_no0_surface) as avgSnow from TEMP_DF group by Geohash order by avgSnow DESC limit 5").collect()
print('Total snowy location count: %d' %len(snow))
for x in snow:
    print(x)
print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Total snowy location count: 5
Row(Geohash='c43k6uu1egxb', avgSnow=0.37908011869436203)
Row(Geohash='c43kcu3t702p', avgSnow=0.37908011869436203)
Row(Geohash='c41ueb1jyypb', avgSnow=0.3775964391691395)
Row(Geohash='c41uhb4r5n00', avgSnow=0.3775964391691395)
Row(Geohash='c41v48pupf00', avgSnow=0.37537091988130566)
Finished. it's been 520 seconds


# Analysis

## Q1. Strangely Snowy: Find a location that contains snow while its surroundings do not. Why does this occur? Is it a high mountain peak in a desert?

In [7]:
df_large.createOrReplaceTempView("TEMP_DF")
not_snow_all_year = spark.sql("SELECT  Geohash,avg(snow_cover_surface) FROM TEMP_DF group by Geohash having avg(snow_cover_surface) >= 99").collect()

for x in not_snow_all_year:
    print(x)
print(len(not_snow_all_year))

Row(Geohash='c1gxsefbz52p', avg(snow_cover_surface)=100.0)
Row(Geohash='c43hd76g0npb', avg(snow_cover_surface)=100.0)
Row(Geohash='c43g3722yrzz', avg(snow_cover_surface)=100.0)
Row(Geohash='c1nt970j5b5b', avg(snow_cover_surface)=100.0)
Row(Geohash='c1grzzxwt1bp', avg(snow_cover_surface)=100.0)
Row(Geohash='c436f3k4p8xb', avg(snow_cover_surface)=100.0)
Row(Geohash='c43jhzrrsbrz', avg(snow_cover_surface)=100.0)
Row(Geohash='c44hk1hy9u2p', avg(snow_cover_surface)=100.0)
Row(Geohash='c46sf3s53p80', avg(snow_cover_surface)=100.0)
Row(Geohash='c43gkzxxn6bp', avg(snow_cover_surface)=100.0)
Row(Geohash='c41tugrtwhzz', avg(snow_cover_surface)=100.0)
Row(Geohash='c458r20zx080', avg(snow_cover_surface)=100.0)
Row(Geohash='c43gb4ywdgrz', avg(snow_cover_surface)=100.0)
Row(Geohash='c438x5esgf00', avg(snow_cover_surface)=100.0)
Row(Geohash='c4459s8rz1xb', avg(snow_cover_surface)=100.0)
Row(Geohash='c43c1zbmntbp', avg(snow_cover_surface)=100.0)
Row(Geohash='c43e1f64v9bp', avg(snow_cover_surface)=100.

In [8]:
four_set = set()
for x in not_snow_all_year:
    print(x.Geohash[0:4])
    four_set.add(x.Geohash[0:4])
print(len(not_snow_all_year))
print(len(four_set))

c1gx
c43h
c43g
c1nt
c1gr
c436
c43j
c44h
c46s
c43g
c41t
c458
c43g
c438
c445
c43c
c43e
c41z
c43u
c44n
c41u
c439
c1q2
c1nt
c432
c43e
c1gr
c44n
c1gn
c43e
c41z
c452
c1nm
c1nr
c44p
c43c
c304
c43b
c460
c43m
c43h
c1gy
c43k
c1gr
c43d
c1gz
c1gx
c43u
c44p
c43d
c41t
c433
c44j
c1q0
c41x
c436
c438
c1pc
c1gx
c43f
c437
c439
c43d
c459
c1nq
c43c
c1nk
c1p5
c460
c436
c43s
c1gz
c1p8
c464
c41v
c3k8
c43k
c45b
c41u
c3hz
c41u
c304
c1pe
c41w
c445
c438
c3k0
c3k3
c43b
c41w
c436
c43h
c460
c43u
c458
c307
c458
c1pf
c437
c464
c44h
c36u
c41g
c44p
c1uh
c44h
c1gy
c43g
c3k2
c461
c41x
c41z
c304
c1nu
c41y
c1gy
c41z
c41w
c301
c36u
c1sn
c43j
c44h
c44w
c43k
c464
c1gq
c1gw
c301
c465
c44n
c43e
c43f
c439
c1nk
c1gz
c439
c1gr
c41v
c43s
c43h
c1sn
c41y
c1gq
c41v
c43f
c4r5
c1p4
c1gz
149
74


In [24]:
# df.createOrReplaceTempView("TEMP_DF")
groupList = list()
for y in four_set:
        param = y + '%'
        group = spark.sql("SELECT distinct Geohash,avg(snow_cover_surface) as avgSnow FROM TEMP_DF WHERE Geohash like '%s' group by Geohash" %param ).collect()
        print(param ) 
        print(len(group))
        groupList.append(group)
        
print(len(groupList))
print('Complete')


c3hz%
4
c43g%
5
c43m%
3
c1p4%
5
c41z%
4
c36u%
4
c1nr%
4
c460%
3
c1p8%
3
c41g%
4
c436%
4
c1nt%
4
c433%
4
c41x%
4
c452%
3
c1q0%
4
c3k8%
4
c43e%
4
c43f%
3
c1uh%
4
c458%
4
c1nm%
3
c41y%
3
c43b%
5
c459%
4
c43u%
4
c1pf%
3
c3k3%
4
c44n%
4
c1gz%
4
c1p5%
3
c44h%
5
c1pc%
4
c1nu%
4
c432%
5
c44j%
4
c1nk%
5
c439%
4
c304%
5
c1gn%
4
c43s%
4
c3k0%
5
c43h%
4
c3k2%
3
c44w%
3
c43k%
4
c41w%
5
c461%
4
c43c%
4
c1gx%
3
c43d%
4
c1gr%
5
c44p%
4
c1nq%
4
c41v%
5
c1gq%
4
c1gw%
4
c41u%
4
c438%
3
c1gy%
4
c1sn%
4
c43j%
2
c41t%
4
c445%
4
c1q2%
5
c4r5%
4
c46s%
4
c45b%
4
c301%
4
c464%
5
c465%
4
c1pe%
4
c307%
4
c437%
4
74
Complete


In [26]:
for x in groupList:
    avg=0
    for y in x:
        avg= avg + y.avgSnow
    avg= avg/len(x)
    if avg<30:
        print(x)
#     print(avg)
        

[Row(Geohash='c1uhvnv5p8pb', avgSnow=7.283950617283951), Row(Geohash='c1uh66r1r02p', avgSnow=9.74074074074074), Row(Geohash='c1uhcd9k1n2p', avgSnow=100.0), Row(Geohash='c1uhqne01400', avgSnow=2.9135802469135803)]
[Row(Geohash='c3k0fgxqugeb', avgSnow=10.790123456790123), Row(Geohash='c3k0yw8m6us0', avgSnow=100.0), Row(Geohash='c3k0rj7e0zs0', avgSnow=21.506172839506174), Row(Geohash='c3k07d6swzkp', avgSnow=4.08641975308642), Row(Geohash='c3k00p3vw3gz', avgSnow=4.08641975308642)]


c1uhcd9k1n2p is a mountain peak(lavender peak)

## Q2. Lightning rod: Where are you most likely to be struck by lightning? Use a precision of at least 4 Geohash characters and provide the top 3 locations.

In [36]:
df_entire.createOrReplaceTempView("TEMP_DF")
started_at = datetime.now()
lightning = spark.sql("SELECT  Geohash,avg(lightning_surface) as avgLighting FROM TEMP_DF group by Geohash order by avgLighting DESC Limit 10 ").collect()

for x in lightning:
    print(x)
print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(Geohash='9eqepuxk7x20', avgLighting=0.3724035608308605)
Row(Geohash='9g3h968ygj7z', avgLighting=0.36053412462908013)
Row(Geohash='9g3y52sgeceb', avgLighting=0.3508902077151335)
Row(Geohash='9g3ug8ckk4hp', avgLighting=0.3486646884272997)
Row(Geohash='9g3m79nrf3zb', avgLighting=0.3479228486646884)
Row(Geohash='9g3mq3f7y6eb', avgLighting=0.3479228486646884)
Row(Geohash='9g3v7kxpuhh0', avgLighting=0.3464391691394659)
Row(Geohash='d7mkkfvu34up', avgLighting=0.336053412462908)
Row(Geohash='9g2vntcg8c2p', avgLighting=0.3353115727002967)
Row(Geohash='9g35cqggdn5z', avgLighting=0.33456973293768544)
10


I calculate the average lightning chances. The top 3 location is 9eqe, 9g3h, 9g3y

## Q3. Drying out: Choose a region in North America (defined by one or more Geohashes) and determine when its driest month is. This should include a histogram with data from each month.

In [57]:
dfList = list()
for x in range(1,13):
    if x<10:
        x = '0'+str(x)
    path = "hdfs://orion11:40910/datasets/nam_2015%s.tdv.gz" %x
#     print(path)
    df_1 = spark.read.format('csv').option('sep', '\t').schema(schema).load(path)
    dfList.append(df_1)
print(len(dfList))

12


In [58]:
#9q9n categorical_rain_yes1_no0_surface
from pyspark.sql.functions import substring
started_at = datetime.now()
for df_1 in dfList:  
    df_1.createOrReplaceTempView("TEMP_DF")
    dry_place = spark.sql("SELECT  substring(Geohash, 1, 4) as subGeo, AVG(categorical_rain_yes1_no0_surface) as avgRain FROM TEMP_DF group by subGeo having subGeo = '9q9n'").collect()
    for x in dry_place:
        print(x)
print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(subGeo='9q9n', avgRain=0.0)
Row(subGeo='9q9n', avgRain=0.0603448275862069)
Row(subGeo='9q9n', avgRain=0.0)
Row(subGeo='9q9n', avgRain=0.0777027027027027)
Row(subGeo='9q9n', avgRain=0.020161290322580645)
Row(subGeo='9q9n', avgRain=0.0022935779816513763)
Row(subGeo='9q9n', avgRain=0.01639344262295082)
Row(subGeo='9q9n', avgRain=0.008333333333333333)
Row(subGeo='9q9n', avgRain=0.009259259259259259)
Row(subGeo='9q9n', avgRain=0.0021551724137931034)
Row(subGeo='9q9n', avgRain=0.0546218487394958)
Row(subGeo='9q9n', avgRain=0.0872093023255814)


This is the avgerage rain chances for Hayward. January, March do not rain at all. [Bay Area](bayArea.jpg)

## Q5. Escaping the fog: After becoming rich from your startup, you are looking for the perfect location to build your Bay Area mansion with unobstructed views. Find the locations that are the least foggy and show them on a map.

In [71]:
started_at = datetime.now()
df_entire.createOrReplaceTempView("TEMP_DF")
visibility = spark.sql("SELECT  Geohash,avg(visibility_surface) as avgVisibility FROM TEMP_DF WHERE Geohash like '9q9p%' or Geohash like '9q9n%' or Geohash like '9q9j%' or Geohash like '9q9m%' or Geohash like '9q9k%' or Geohash like '9q9h%' or Geohash like '9q8v%' or Geohash like '9q8y%' or Geohash like '9q8z%' group by Geohash order by avgVisibility DESC ").collect()
for x in visibility:
    print(x)
print(len(visibility))
print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

Row(Geohash='9q9jnpfx5rxb', avgVisibility=23811.522961255705)
Row(Geohash='9q9jv7p453zz', avgVisibility=23703.065985376827)
Row(Geohash='9q9m35ffeg7z', avgVisibility=23698.021476396112)
Row(Geohash='9q9kc8wprss0', avgVisibility=23675.469547196735)
Row(Geohash='9q9jdrny12rz', avgVisibility=23616.419106960297)
Row(Geohash='9q8zuzx3hzeb', avgVisibility=23600.692098283627)
Row(Geohash='9q9k4m9efqh0', avgVisibility=23497.428004600173)
Row(Geohash='9q9hp3c2xy5b', avgVisibility=23449.653520560052)
Row(Geohash='9q9hwsyh477z', avgVisibility=23436.152038374712)
Row(Geohash='9q9mbqzmqxrz', avgVisibility=23435.8553033342)
Row(Geohash='9q9pesk2gbs0', avgVisibility=23400.098629552464)
Row(Geohash='9q9j5h47k07z', avgVisibility=23329.623851402575)
Row(Geohash='9q9py8hurvup', avgVisibility=23277.991815833026)
Row(Geohash='9q9nky5sd52p', avgVisibility=23164.93542607814)
Row(Geohash='9q9nxfh005kp', avgVisibility=23088.525926889935)
Row(Geohash='9q9p887u91up', avgVisibility=22978.882014243467)
Row(Geohash

The list above has the best visibility in the bay area. 
Such as 9q9m35 is a good place to build a mansion. The address is S Grimmer Blvd:Fremont Blvd, Fremont, CA
[Grimmer](Grimmer.jpg)

# Option 1

In [120]:
df_option = spark.read.option("header","true").option("inferSchema","true").csv('hdfs://orion11:40910/option_dataset/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')


In [118]:
notComplete = df_option.filter(df_option.Category == "ASSAULT").count()
print(notComplete)

194694


In [122]:
df_option.createOrReplaceTempView("POLICE_DF")
selectAll = spark.sql("select Category, Date  from POLICE_DF limit 5").show()
# print(selectAll)


+---------------+----------+
|       Category|      Date|
+---------------+----------+
|   NON-CRIMINAL|01/19/2015|
|        ROBBERY|02/01/2015|
|        ASSAULT|02/01/2015|
|SECONDARY CODES|02/01/2015|
|      VANDALISM|01/27/2015|
+---------------+----------+



## Q1. 2003-2017, 1- 24. batman

In [ ]:
started_at = datetime.now()
df_entire.createOrReplaceTempView("TEMP_DF")
visibility = spark.sql("SELECT  Geohash,avg(visibility_surface) as avgVisibility FROM TEMP_DF WHERE Geohash like '9q9p%' or Geohash like '9q9n%' or Geohash like '9q9j%' or Geohash like '9q9m%' or Geohash like '9q9k%' or Geohash like '9q9h%' or Geohash like '9q8v%' or Geohash like '9q8y%' or Geohash like '9q8z%' group by Geohash order by avgVisibility DESC ").collect()
for x in visibility:
    print(x)
print(len(visibility))
print("Finished. it's been " + str((datetime.now()-started_at).seconds) + " seconds")

## Q2. Break in car (VANDALISM) when and where

## Q3. 
### a. Top 10 Category and resolution rate. 

### b. Relationship between criminal increase rate and solved criminal rate